# --- Day 7: The Sum of Its Parts ---

You find yourself standing on a snow-covered coastline; apparently, you landed a little off course. The region is too hilly to see the North Pole from here, but you do spot some Elves that seem to be trying to unpack something that washed ashore. It's quite cold out, so you decide to risk creating a paradox by asking them for directions.

"Oh, are you the search party?" Somehow, you can understand whatever Elves from the year 1018 speak; you assume it's Ancient Nordic Elvish. Could the device on your wrist also be a translator? "Those clothes don't look very warm; take this." They hand you a heavy coat.

"We do need to find our way back to the North Pole, but we have higher priorities at the moment. You see, believe it or not, this box contains something that will solve all of Santa's transportation problems - at least, that's what it looks like from the pictures in the instructions." It doesn't seem like they can read whatever language it's in, but you can: "Sleigh kit. Some assembly required."

"'Sleigh'? What a wonderful name! You must help us assemble this 'sleigh' at once!" They start excitedly pulling more parts out of the box.

The instructions specify a series of steps and requirements about which steps must be finished before others can begin (your puzzle input). Each step is designated by a single letter. For example, suppose you have the following instructions:

```
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
```

Visually, these requirements look like this:

```
  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
```

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

Only C is available, and so it is done first.
Next, both A and F are available. A is first alphabetically, so it is done next.
Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.
F is the only choice, so it is done next.
Finally, E is completed.
So, in this example, the correct order is `CABDFE`.

In what order should the steps in your instructions be completed?

In [7]:
import re
from dataclasses import dataclass, field

In [1]:
def test_input():
    yield 'Step C must be finished before step A can begin.'
    yield 'Step C must be finished before step F can begin.'
    yield 'Step A must be finished before step B can begin.'
    yield 'Step A must be finished before step D can begin.'
    yield 'Step B must be finished before step E can begin.'
    yield 'Step D must be finished before step E can begin.'
    yield 'Step F must be finished before step E can begin.'

In [2]:
def input_():
    with open('./inputs/day07/input.txt') as fp:
        yield from (line.strip() for line in fp)

In [4]:
STEP_RE = re.compile(r'^Step (?P<from>\w) must be finished before step (?P<to>\w) can begin.$')

In [5]:
STEP_RE.match('Step C must be finished before step A can begin.')

<re.Match object; span=(0, 48), match='Step C must be finished before step A can begin.'>

In [53]:
@dataclass
class Node:
    id_: str
    done: bool = field(init=False, default=False)
    duration: int = field(init=False, default=0)
    children: set = field(init=False, default_factory=set)
    parents: set = field(init=False, default_factory=set)
    
    def add_child(self, child):
        self.children.add(child)
    
    def add_parent(self, parent):
        self.parents.add(parent)
    
    def __hash__(self):
        return hash(self.id_)

    @property
    def ready(self):
        return all(node.done for node in self.parents)
    
    def step_time(self, test=False):
        offset = 64 if test else 4
        return ord(self.id_) - offset

In [60]:
@dataclass
class Graph:
    nodes: dict = field(init=False, default_factory=dict, repr=False)
    
    @classmethod
    def from_steps(cls, steps):
        graph = cls()
        for step in steps:
            result = STEP_RE.match(step)
            graph.add_edge(result['from'], result['to'])
        return graph
        
    def add_edge(self, from_, to):
        self[from_].add_child(self[to])
        self[to].add_parent(self[from_])
    
    def __getitem__(self, key):
        return self.nodes.setdefault(key, Node(key))
    
    def next(self):
        ready_nodes = sorted(
            (node for node in self.nodes.values() if node.ready and not node.done),
            key=lambda node: node.id_)
        
        if not ready_nodes:
            return None
        
        next_node = ready_nodes[0]
        next_node.done = True
        
        return next_node
    
    def __iter__(self):
        return iter(self.next, None)
    
    def process_time(self, num_workers, test=False):
        counter = 0
        active_steps = set()
        
        while True:
            # mark off any steps that are done
            done_steps = set()
            for step in active_steps:
                if step.duration == step.step_time(test):
                    step.done = True
                    done_steps.add(step)
            active_steps -= done_steps
            
            # find any nodes that are not done, are ready, and are not already active
            ready_nodes = sorted(
                (node for node in self.nodes.values() if not node.done and node.ready and node.duration == 0),
                key=lambda node: node.id_)
            
            if not active_steps and not ready_nodes:
                break
                
            # add new steps if there are any ready and there are workers available
            if len(active_steps) < num_workers and ready_nodes:
                active_steps.update(ready_nodes[:(num_workers - len(active_steps))])
            
            for step in active_steps:
                step.duration += 1
            
            counter += 1
        
        return counter

In [37]:
''.join(node.id_ for node in Graph.from_steps(test_input()))

'CABDFE'

In [38]:
''.join(node.id_ for node in Graph.from_steps(input_()))

'CGKMUWXFAIHSYDNLJQTREOPZBV'

# --- Part Two ---

As you're about to begin construction, four of the Elves offer to help. "The sun will set soon; it'll go faster if we work together." Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

```
Second   Worker 1   Worker 2   Done
   0        C          .        
   1        C          .        
   2        C          .        
   3        A          F       C
   4        B          F       CA
   5        B          F       CA
   6        D          F       CAB
   7        D          F       CAB
   8        D          F       CAB
   9        D          .       CABF
  10        E          .       CABFD
  11        E          .       CABFD
  12        E          .       CABFD
  13        E          .       CABFD
  14        E          .       CABFD
  15        .          .       CABFDE
```

Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or `.` if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

In [41]:
def step_time(id_, test=False):
    offset = 64 if test else 4
    return ord(id_) - offset

In [39]:
ord('A'), ord('Z')

(65, 90)

In [43]:
step_time('A'), step_time('Z')

(61, 86)

In [44]:
step_time('A', True), step_time('Z', True)

(1, 26)

In [61]:
Graph.from_steps(test_input()).process_time(num_workers=2, test=True)

15

In [62]:
# Should be less than 1048
Graph.from_steps(input_()).process_time(num_workers=5, test=False)

1046